In [ ]:

# 🎓 FC25 Sentiment Training Notebook

# ✅ STEP 1: Upload Labeled Data
from google.colab import files
uploaded = files.upload()

# Upload one or both of the following:
# - labeled_comments.csv
# - corrected_sentiment_batch.csv

# ✅ STEP 2: Load and Merge Labeled Data
import pandas as pd

df_list = []

if "labeled_comments.csv" in uploaded:
    df1 = pd.read_csv("labeled_comments.csv")
    if "Corrected Sentiment" in df1.columns:
        df1 = df1.rename(columns={"Corrected Sentiment": "Sentiment"})
    df_list.append(df1)

if "corrected_sentiment_batch.csv" in uploaded:
    df2 = pd.read_csv("corrected_sentiment_batch.csv")
    if "Corrected Sentiment" in df2.columns:
        df2 = df2.rename(columns={"Corrected Sentiment": "Sentiment"})
    df_list.append(df2)

if not df_list:
    raise Exception("Please upload at least one labeled data file.")

df = pd.concat(df_list, ignore_index=True)
df = df[df["Sentiment"].isin(["Positive", "Negative", "Neutral"])]
df = df.drop_duplicates(subset=["Comment"])

# ✅ STEP 3: Train Model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

X_train, X_test, y_train, y_test = train_test_split(df["Comment"], df["Sentiment"], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=3000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression(max_iter=200)
model.fit(X_train_vec, y_train)

# ✅ STEP 4: Evaluate
y_pred = model.predict(X_test_vec)
print("Model Evaluation Report:")
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# ✅ STEP 5: Export Trained Model and Vectorizer
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

files.download("model.pkl")
files.download("vectorizer.pkl")
